In [2]:
from utils import load_caption, load_vocab, decode_caption, load_annotations
from nltk.translate import bleu_score
import os
from tqdm import tqdm
import pickle

import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
vocab = load_vocab('/home/spb61/coco2014_vocab.json')
image_id_to_index, index_to_image_id, annotations_dict = load_annotations(annotations_dir='/home/spb61/annotations',
                                                                          annotations_file='captions_val2014.json',
                                                                          map_file = '/home/spb61/val_image_id_to_idx.csv')

word_to_idx
idx_to_word
Loaded dictionary...
Dictionary size: 5004
Error proccessing image_id: image_index
Skipping file person_keypoints_train2014.json
Skipping file instances_train2014.json


In [4]:
def get_annotations(image_id):
    return [' '.join(cap) for cap in annotations_dict[image_id]]

In [5]:
def save_beams_to_pickle(k):
    captions = {}
    image_dir = "/datadrive/val_beam_{}_states/".format(k)
    for image in tqdm(sorted(os.listdir(image_dir))):
        caption_object = load_caption(image, image_dir=image_dir)
        captions[caption_object['image_id']] = [{
            'score': cap['score'],
            'sentence': cap['sentence']
        } for cap in caption_object['captions']]
    print("Images found: {}".format(len(captions)))
    pickle.dump(captions, open("/datadrive/beams_{}.pickle".format(k), 'wb'))

In [27]:
beam_caps = []
references = []

image_dir = "/datadrive/val_beam_{}_states/".format(1)
for image in tqdm(sorted(os.listdir(image_dir))):
    image_id = int(image.split(".")[0])
    caption_object = load_caption(image, image_dir=image_dir)
    caption = ' '.join(decode_caption(caption_object['captions'][0]['sentence'], vocab))
    beam_caps.append(caption)
    annotations = [' '.join(c) for c in annotations_dict[image_id]]
    references.append(annotations)

100%|██████████| 40504/40504 [07:06<00:00, 94.87it/s]


In [32]:
cat /datadrive/val_beam_10_states/1.json

{"probabilities": [], "captions": [{"score": 0.0029771133116083663, "hidden_state_average": [[1.0096803411467072, -0.2759297359734355, 0.22449681383662182, 0.09498238773085177, -0.32089411737804313, 0.11443533301353703, 0.005791841173004286, 0.2118074357509613, -0.04515499118715525, -0.15927147082984447, 0.1439541310071945, -0.04185029938817024, -0.44602605402469636, 0.3924172967672348, -0.39459448158740995, 0.13755004703998566, 1.4606734938919543, -0.9288065537810326, -0.3934544760733843, -1.4777392387390136, -1.4867025017738342, -0.302975033596158, -0.13847411144524813, 0.2998923555016518, -0.12687679308677105, -0.30876212418079374, 0.14407851384021342, -0.07596253752708435, -0.5202307782541971, 0.4560111999511718, 0.6340057492256165, -0.727834356483072, 0.6960775494575501, -0.1002450168132782, 0.5867850668728352, 0.3337356302887201, -0.8700348287820816, 0.361907371878624, -0.6479766130447387, 0.07231648331508041, -0.2462387246079743, -0.3678454544395208, 0.21117878258228293, -0.4502

In [29]:
from nltk.translate import bleu_score
bleu_score.corpus_bleu(references, beam_caps)

0.6287890108864951

In [ ]:
save_beams_to_pickle(100)

In [ ]:
%ls -lh /datadrive
captions = pickle.load(open("/datadrive/beams_{}.pickle".format(10), 'rb'))

In [12]:
captions

{0: [{'score': 0.008501108008349322,
   'sentence': [1, 4, 13, 47, 4, 92, 36, 7, 70, 6, 4, 363, 2]},
  {'score': 0.004674439808604864,
   'sentence': [1, 4, 13, 11, 261, 4, 286, 5, 4, 92, 2]},
  {'score': 0.004409960379403454,
   'sentence': [1, 4, 31, 287, 4, 251, 120, 8, 7, 125, 2]},
  {'score': 0.004299761811615586,
   'sentence': [1, 4, 13, 47, 4, 92, 33, 7, 70, 6, 4, 363, 2]},
  {'score': 0.0013523909810635856,
   'sentence': [1, 4, 13, 261, 4, 286, 5, 4, 92, 2]},
  {'score': 0.0012689868894483884,
   'sentence': [1, 4, 13, 261, 4, 286, 5, 4, 92, 5, 4, 363, 2]},
  {'score': 0.0010182044895239498,
   'sentence': [1, 4, 13, 47, 4, 92, 5, 4, 363, 2]},
  {'score': 0.0010085788420457912,
   'sentence': [1, 4, 13, 47, 4, 92, 33, 4, 363, 2]},
  {'score': 0.0008947542249237514,
   'sentence': [1, 4, 31, 287, 4, 251, 120, 8, 7, 125, 12, 2]},
  {'score': 0.0008270562976665659,
   'sentence': [1, 4, 13, 261, 4, 286, 5, 4, 92, 18, 4, 251, 139, 2]}],
 1: [{'score': 0.0029771133116083663,
   's